In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from io import StringIO
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.feature_selection import RFE

from sklearn.metrics import confusion_matrix
from keras.layers import Dense, Activation, Dropout
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import random
import openpyxl


from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.utils.class_weight import compute_class_weight
from itertools import combinations
import joblib

In [2]:
df1 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/PhoneLog/raw_phonelog_features_studentlife.csv')
df1 = df1.rename(columns={'Date': 'date'})
df1 = df1.drop('Unnamed: 0', axis=1)
df2 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Stress/recreating_dailystress_features.csv')
df2 = df2.drop('Unnamed: 0', axis=1)

In [3]:
df = pd.merge(df1, df2, on=['uid', 'date'])
print(df.columns)

df = df.sort_values(by='date')

Index(['date', 'uid', 'morning_phonecharge_duration',
       'morning_phonecharge_frequency', 'morning_phonelock_duration',
       'morning_phonelock_frequency', 'morning_dark_duration',
       'afternoon_phonecharge_duration', 'afternoon_phonecharge_frequency',
       'afternoon_phonelock_duration', 'afternoon_phonelock_frequency',
       'afternoon_dark_duration', 'evening_phonecharge_duration',
       'evening_phonecharge_frequency', 'evening_phonelock_duration',
       'evening_phonelock_frequency', 'evening_dark_duration',
       'night_phonecharge_duration', 'night_phonecharge_frequency',
       'night_phonelock_duration', 'night_phonelock_frequency',
       'night_dark_duration', 'stress_ratings'],
      dtype='object')


In [4]:
df.isnull().sum()

date                                 0
uid                                  0
morning_phonecharge_duration       205
morning_phonecharge_frequency      205
morning_phonelock_duration           5
morning_phonelock_frequency          5
morning_dark_duration               47
afternoon_phonecharge_duration       0
afternoon_phonecharge_frequency      0
afternoon_phonelock_duration         0
afternoon_phonelock_frequency        0
afternoon_dark_duration              0
evening_phonecharge_duration         5
evening_phonecharge_frequency        5
evening_phonelock_duration           0
evening_phonelock_frequency          0
evening_dark_duration                0
night_phonecharge_duration           5
night_phonecharge_frequency          5
night_phonelock_duration             0
night_phonelock_frequency            0
night_dark_duration                  0
stress_ratings                       0
dtype: int64

In [5]:
df = df.dropna()
print(len(df))

3349


In [6]:
df.columns

Index(['date', 'uid', 'morning_phonecharge_duration',
       'morning_phonecharge_frequency', 'morning_phonelock_duration',
       'morning_phonelock_frequency', 'morning_dark_duration',
       'afternoon_phonecharge_duration', 'afternoon_phonecharge_frequency',
       'afternoon_phonelock_duration', 'afternoon_phonelock_frequency',
       'afternoon_dark_duration', 'evening_phonecharge_duration',
       'evening_phonecharge_frequency', 'evening_phonelock_duration',
       'evening_phonelock_frequency', 'evening_dark_duration',
       'night_phonecharge_duration', 'night_phonecharge_frequency',
       'night_phonelock_duration', 'night_phonelock_frequency',
       'night_dark_duration', 'stress_ratings'],
      dtype='object')

In [7]:
len(df.columns)

23

In [8]:
df['stress_ratings'].value_counts()

stress_ratings
medium stress    1328
high stress      1043
low stress        978
Name: count, dtype: int64

In [9]:
binary_lh_data = df[df['stress_ratings'].isin(['low stress', 'high stress'])]

In [10]:
print(len(binary_lh_data['uid'].unique()))
binary_lh_data['uid'].value_counts()

43


uid
u59    152
u16    128
u19    126
u49    111
u04     97
u33     87
u57     84
u58     82
u08     79
u00     71
u52     68
u46     60
u44     54
u45     54
u22     48
u12     48
u32     47
u01     43
u02     40
u07     39
u36     38
u51     37
u30     35
u24     34
u35     32
u27     32
u53     31
u25     30
u54     30
u42     29
u14     28
u56     27
u31     22
u23     21
u18     16
u17     12
u41     12
u47     11
u05      9
u34      5
u15      5
u20      4
u09      3
Name: count, dtype: int64

In [11]:
# uid_counts = binary_lh_data['uid'].value_counts()
# uids_to_keep = uid_counts[uid_counts >= 5].index
# binary_lh_data = binary_lh_data[binary_lh_data['uid'].isin(uids_to_keep)]

# print('Length of Data: ', len(binary_lh_data))
# binary_lh_data = binary_lh_data.sort_values(by=['uid', 'date'])
# print(binary_lh_data['uid'].unique())

In [12]:
X = binary_lh_data.drop(columns=['stress_ratings', 'uid', 'date'])
y = binary_lh_data['stress_ratings']
groups = binary_lh_data['uid']

stress_map = {'low stress': 0, 'high stress': 1}
y_encoded = y.map(stress_map).values 

In [13]:
print(f"Number of features in X: {X.shape[1]}")

Number of features in X: 20


In [14]:
logo = LeaveOneGroupOut()

best_thresholds = []
balanced_accs = []
auc_scores = []

In [15]:
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

In [16]:
param_grid = [
    {'C': [0.001, 0.01, 0.1, 1, 10, 100],
     'penalty': ['l2'],
     'solver': ['newton-cg', 'lbfgs', 'sag']},
    {'C': [0.001, 0.01, 0.1, 1, 10, 100],
     'penalty': ['l1', 'l2'],
     'solver': ['liblinear']},
    {'C': [0.001, 0.01, 0.1, 1, 10, 100],
     'l1_ratio': [0.2, 0.5, 0.8],
     'penalty': ['elasticnet'],
     'solver': ['saga']}
]

In [17]:
output_list = []
scaler = StandardScaler()
model_logistic_rfe = LogisticRegression(max_iter=5000)

rfe = RFE(model_logistic_rfe, n_features_to_select=15)  
X_ = scaler.fit_transform(X)
rfe.fit(X_, y_encoded) 

best_features = np.where(rfe.support_)[0]
print(f"Selected Features: {best_features}")

num_splits = len(np.unique(groups))
print(num_splits)

with tqdm(total=num_splits, desc="LOSO CV Progress", unit="fold", bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt} [{elapsed}<{remaining}] {percentage:3.0f}%") as pbar:
    for train_idx, test_idx in logo.split(X, y_encoded, groups=groups):
        output_capture = StringIO()
        X_train, X_test = X.iloc[train_idx, best_features], X.iloc[test_idx, best_features]
        y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]

        print('X and y train shapes: ', file=output_capture)
        print(X_train.shape, file=output_capture)
        print(y_train.shape, file=output_capture)

        print('X and y test shapes: ', file=output_capture)
        print(X_test.shape, file=output_capture)
        print(y_test.shape, file=output_capture)

        stress_labels = {0: 'low stress', 1: 'high stress'}
        print('y_train class distribution:', {stress_labels[k]: v for k, v in dict(zip(*np.unique(y_train, return_counts=True))).items()}, file=output_capture)
        print('y_test class distribution:', {stress_labels[k]: v for k, v in dict(zip(*np.unique(y_test, return_counts=True))).items()}, file=output_capture)

        scaler = StandardScaler()
        X_train_normalized = scaler.fit_transform(X_train)
        X_test_normalized = scaler.transform(X_test)
        
        model_logistic = GridSearchCV(LogisticRegression(max_iter=5000, class_weight='balanced'),
                                      param_grid=param_grid, cv=3, verbose=True, n_jobs=-1, scoring='roc_auc')
        if len(np.unique(y_train)) > 1 and len(np.unique(y_test)) > 1:
            model_logistic.fit(X_train_normalized, y_train)
    
            y_test_pred_proba = model_logistic.predict_proba(X_test_normalized)[:, 1]
            thresholds = np.arange(0.01, 1.0, 0.01)
            best_threshold = 0.5
            best_metric = 0.0
    
            for threshold in thresholds:
                y_test_pred_binary = (y_test_pred_proba > threshold).astype(int)
                metric_value = balanced_accuracy_score(y_test, y_test_pred_binary)
                if metric_value > best_metric:
                    best_metric = metric_value
                    best_threshold = threshold
            if len(np.unique(y_test)) > 1:
                auc_score = roc_auc_score(y_test, y_test_pred_proba)
                auc_scores.append(auc_score)
                print(f"AUC Score: {auc_score}", file=output_capture)
            else:
                auc_scores.append(None)
                print(f"Skipping AUC computation for this fold as y_test contains only one class: {np.unique(y_test)}")
    
            y_test_pred_binary = (y_test_pred_proba > best_threshold).astype(int)
            balanced_acc = balanced_accuracy_score(y_test, y_test_pred_binary)
            balanced_accs.append(balanced_acc)
        
            print(f"Balanced Accuracy: {balanced_acc}", file=output_capture)
            print(f"y_test (True): {y_test}", file=output_capture)
            print(f"y_predicted (Pred): {y_test_pred_binary}", file=output_capture)

            y_train_pred_proba = model_logistic.predict_proba(X_train_normalized)[:, 1]
            y_test_pred_proba = model_logistic.predict_proba(X_test_normalized)[:, 1]

            accuracy_train = accuracy_score(y_train, (y_train_pred_proba > best_threshold).astype(int))
            accuracy_test = accuracy_score(y_test, (y_test_pred_proba > best_threshold).astype(int))
            
            print(f"Training Accuracy: {accuracy_train}", file=output_capture)
            print(f"Test Accuracy: {accuracy_test}", file=output_capture)

        else:
            print(f"Skipping this subject for having single class: y_train = {np.unique(y_train)}, y_test = {np.unique(y_test)}")
        
        pbar.update(1)
        output_list.append(output_capture.getvalue())

Selected Features: [ 2  3  5  6  7  8  9 10 11 12 13 14 15 16 18]
43


LOSO CV Progress:   0%|                                    | 0/43 [00:00<?]   0%

Fitting 3 folds for each of 48 candidates, totalling 144 fits


LOSO CV Progress:   2%|▋                               | 1/43 [00:01<00:43]   2%

Fitting 3 folds for each of 48 candidates, totalling 144 fits


...

LOSO CV Progress:  95%|████████████████████████████▌  | 41/43 [00:11<00:00]  95%

Fitting 3 folds for each of 48 candidates, totalling 144 fits


LOSO CV Progress:  98%|█████████████████████████████▎ | 42/43 [00:12<00:00]  98%

Fitting 3 folds for each of 48 candidates, totalling 144 fits


LOSO CV Progress: 100%|██████████████████████████████ | 43/43 [00:12<00:00] 100%


In [18]:
output_list

["X and y train shapes: \n(1950, 15)\n(1950,)\nX and y test shapes: \n(71, 15)\n(71,)\ny_train class distribution: {'low stress': 948, 'high stress': 1002}\ny_test class distribution: {'low stress': 30, 'high stress': 41}\nAUC Score: 0.5166666666666666\nBalanced Accuracy: 0.5707317073170732\ny_test (True): [1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]\ny_predicted (Pred): [0 1 0 0 1 1 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 1 0\n 1 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1]\nTraining Accuracy: 0.5358974358974359\nTest Accuracy: 0.5352112676056338\n",
 "X and y train shapes: \n(1978, 15)\n(1978,)\nX and y test shapes: \n(43, 15)\n(43,)\ny_train class distribution: {'low stress': 952, 'high stress': 1026}\ny_test class distribution: {'low stress': 26, 'high stress': 17}\nAUC Score: 0.5701357466063348\nBalanced Accuracy: 0.6119909502262444\ny_test (

In [19]:
df_output = pd.DataFrame({'Fold Results in PhoneLog Features': output_list})
df_output.to_excel('studentlife_phonelog_prediction_analysis.xlsx', index=False)

In [20]:
print('Logistic Regression: ')
print('Median, 25th Percentile, 75th Percentile: ')

if auc_scores:
    auc_scores_valid = [score for score in auc_scores if score is not None] 

    auc_median = np.median(auc_scores_valid)
    auc_25_percentile = np.percentile(auc_scores_valid, 25)
    auc_75_percentile = np.percentile(auc_scores_valid, 75)
    print(f"AUC Score - Median: {auc_median:.4f}, 25th Percentile: {auc_25_percentile:.4f}, 75th Percentile: {auc_75_percentile:.4f}")

balanced_accs_valid = [acc for acc in balanced_accs if acc is not None] 

balanced_acc_median = np.median(balanced_accs)
balanced_acc_25_percentile = np.percentile(balanced_accs, 25)
balanced_acc_75_percentile = np.percentile(balanced_accs, 75)

print(f"Balanced Accuracy - Median: {balanced_acc_median:.4f}, 25th Percentile: {balanced_acc_25_percentile:.4f}, 75th Percentile: {balanced_acc_75_percentile:.4f}")

Logistic Regression: 
Median, 25th Percentile, 75th Percentile: 
AUC Score - Median: 0.5637, 25th Percentile: 0.5189, 75th Percentile: 0.6056
Balanced Accuracy - Median: 0.6120, 25th Percentile: 0.5597, 75th Percentile: 0.6408
